# Collaborative Filtering

In [47]:
import os
import sys

sys.path.append("/Users/eric/Code/recommender-system/src")

import csv
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

from data.db import Recommendations
from data.db import insert

NB_USERS = 50000
NB_ITEMS = 200000

In [24]:
X = sparse.lil_matrix((NB_USERS, NB_ITEMS))

In [25]:
with open("../datasets/movielens/ml-20m/ratings.csv", "r") as f:
    reader = csv.reader(f, delimiter=',')
    next(reader, None)  # skip header

    for i, (user_id, movie_id, rating, timestamp) in enumerate(reader):
        X[user_id, movie_id] = rating
        
        if i == 10000:
            break

### Cosine Similarity on Items

In [41]:
item_similarities = cosine_similarity(X.T)

In [ ]:
to_insert = []

for i in range(len(item_similarities)):
    for index, score in enumerate(similarities[i]):
        if score < 0.5 or i == index:
            continue
            
        r = Recommendations(**{"engine_name": "CosineMovies",
                               "source_product_id": i,
                               "recommended_product_id": index,
                               "score": score})
    
        to_insert.append(r)

In [ ]:
len(to_insert)

In [64]:
insert(to_insert)